In [1]:
import pandas as pd
import numpy as np
import h5py
import os

# Households

In [3]:
base_dir = r'\\aws-model05\e$\activitysim\activitysim\examples'

In [73]:
df_mtc = pd.read_csv(os.path.join(base_dir,r'example_mtc','data','households.csv'))
syn_hh = pd.read_csv(r'R:\e2projects_two\2018_base_year\synthetic_population\final_output\adjusted_synthetic_households.csv')

In [4]:
hh_persons = h5py.File(r'R:\e2projects_two\SoundCast\Inputs\dev\landuse\2018\base_year\hh_and_persons.h5', 'r')
df_psrc = pd.DataFrame()
for col in hh_persons['Household'].keys():
    df_psrc[col] = hh_persons['Household'][col][:]
    
df_psrc_person = pd.DataFrame()
for col in hh_persons['Person'].keys():
    df_psrc_person[col] = hh_persons['Person'][col][:]
    
hh_persons.close()

In [5]:
# df_psrc = df_psrc.iloc[0:len(df_mtc)]
# df_psrc_person = df_psrc_person[df_psrc_person['hhno'].isin(df_psrc['hhno'])]

In [74]:
# Join missing data to household file from seed HH file (PUMS)
# load_cols = ['hhnum','HHT', 'NOC', 'SERIALNO','TYPE','VEH',
#             'PUMA5','TEN','WIF']
seed_hh = pd.read_csv(r'R:\e2projects_two\2018_base_year\synthetic_population\adjusted_seed_households.csv')


In [75]:
df_psrc = df_psrc.merge(syn_hh[['household_id','hhnum']], left_on='hhno', right_on='household_id')


In [76]:

# get col list required from PUMS


df_psrc = df_psrc.merge(seed_hh, on='hhnum', how='left')

df_psrc.rename(columns={'hhtaz': 'TAZ',
                           'hhincome': 'income',
                            'hhsize': 'PERSONS',
                           'VEH': 'VEHICL',
                           'TYPE': 'UNITTYPE',
                        'BLD': 'BLDGSZ',
                       'PUMA': 'PUMA5',
                        'TEN':'TENURE',
                        'WIF': 'hwrkrcat'}, inplace=True)
df_psrc['HHID'] = df_psrc['hhno'].copy()

In [77]:
# Houeshold income categorization number 
# Integer, 1 - 0 to 20k(-); 
# 2 - 20 to 50k;
# 3 - 50 to 100k; 
# 4 - more than 100k
df_psrc['hinccat1'] = pd.cut(df_psrc['income'], 
                                 [0,20000,50000,100000,99999999999999],
                                labels=[1,2,3,4])

df_psrc['hinccat2'] = pd.cut(df_psrc['income'], 
                                 [0,10000,20000,30000,40000,
                                  50000, 60000,75000, 100000, 99999999999999],
                                labels=[1,2,3,4,5,6,7,8,9])

# num workers
_df = df_psrc_person[df_psrc_person['pwtyp'] >0].groupby('hhno').count()[['psexpfac']].reset_index()
_df.rename(columns={'psexpfac': 'workers'}, inplace=True)
df_psrc = df_psrc.merge(_df, on='hhno', how='left')
df_psrc['workers'].fillna(0, inplace=True)

In [78]:
len(df_psrc)

1605803

In [79]:




# Create other categories 
# hsizecat (houeshold size category, 1, 2, 3, 4+)
df_psrc['hsizecat'] = df_psrc['PERSONS'].copy()
df_psrc.loc[df_psrc['hsizecat']  >= 4, 'hsizecat'] = 4

# From person PUMS files
# hhagecat (head of hoeshold age category)

# hfamily

# hunittype
df_psrc['hunittype'] = df_psrc['UNITTYPE'].copy()

# hNOCcat (number of children category)
df_psrc['hNOCcat'] = 0
df_psrc.loc[df_psrc['NOC'] >= 1, 'hhNOCcat'] = 1

# h0004 (int for members 0 to 4)
# h0511
# h1215
# h1617
# h1824
# h2534
# h3549
# h5064
# h6579
# h80+

age_cols = ['h004','h0511','h1215','h1617',
            'h1824','h2534','h3549','h5064',
            'h6579','h80up']
df_psrc_person['age_category'] = pd.cut(df_psrc_person['pagey'], 
                                 [-1,4,11,15,17,24,34,49,64,79,999],
                                 labels=age_cols)

In [80]:
# df = df_psrc_person.groupby(['hhno','age_category']).size().reset_index()
# _df = df.loc[df['age_category'] == age_cols[0]]
# _df

In [81]:


df = df_psrc_person.groupby(['hhno','age_category']).size().reset_index()
# dfnew = df_psrc.copy()
for age_category in age_cols:
    _df = df.loc[df['age_category'] == age_category]
    df_psrc = df_psrc.merge(_df[['hhno',0]], on='hhno', how='left')
    df_psrc[0] = df_psrc[0].fillna(0).astype('int')
    df_psrc.rename(columns={0: age_category}, inplace=True)

# hwork_f
# hwork_p
# huniv
# hnwork
# hretire
# hpresch
# hschpred
# hschdriv
# Create a recode for employment type
df_psrc_person['mtc_worker_type'] = df_psrc_person['pptyp'].copy()
df_psrc_person['mtc_worker_type'] = df_psrc_person['mtc_worker_type'].map(
                            {1:'hwork_f',
                              2:'hwork_p',
                              5:'huniv',
                               4:'hnwork',
                               3:'hretire',
                               8:'hpresch',
                               7:'hschpred',
                               6:'hschdriv',
                              })

df = df_psrc_person.groupby(['hhno','mtc_worker_type']).size().reset_index()
mtc_worker_cats = ['hwork_f','hwork_p','huniv','hnwork','hretire','hrpresch',
                  'hschpred','hschdriv']

for worker_cat in mtc_worker_cats:
#     print(worker_cat)
    _df = df.loc[df['mtc_worker_type'] == worker_cat]
    df_psrc = df_psrc.merge(_df[['hhno',0]],how='left', on='hhno')
    df_psrc[0] = df_psrc[0].fillna(0).astype('int')
    df_psrc.rename(columns={0: age_category}, inplace=True)


# htypdwel (1 SFH detached, 2 duplex or apt, 3 mobile home etc)
# df_psrc['htypdwel'] = df_psrc['BLDGSZ'].copy()
df_psrc.loc[df_psrc['BLDGSZ'].isin([2,3]), 'htypdwel'] = 1
df_psrc.loc[df_psrc['BLDGSZ'].isin([4,5,6,7,8,9]), 'htypdwel'] = 2
df_psrc.loc[df_psrc['BLDGSZ'].isin([1,10]), 'htypdwel'] = 3

In [82]:
len(df_psrc)

1605803

In [83]:


# hownrent (same as in soundcast)

# hadnwst (nonworking students in household)
# hadwpst

# student nonworker
df_psrc_person['hadnwst'] = 0
df_psrc_person.loc[(df_psrc_person['pstyp'] > 0) 
                    & (df_psrc_person['pwtyp'] == 0),'hadnwst'] = 1
# student worker
# df_psrc_person[(df_psrc_person['student'] > 0) & df_psrc_person['pwtyp'] > 0]
df_psrc_person['hadwpst'] = 0
df_psrc_person.loc[(df_psrc_person['pstyp'] > 0) 
                    & (df_psrc_person['pwtyp'] > 0),'hadwpst'] = 1

df = df_psrc_person.groupby('hhno').sum()[['hadwpst']].reset_index()
df.loc[df['hadwpst'] > 0, 'hadwpst'] = 1
df_psrc = df_psrc.merge(df[['hhno','hadwpst']],how='left', on='hhno')
# df_psrc.drop('hhno', axis=1, inplace=True)

df = df_psrc_person.groupby('hhno').sum()[['hadnwst']].reset_index()
df.loc[df['hadnwst'] > 0, 'hadnwst'] = 1
df_psrc = df_psrc.merge(df[['hhno','hadnwst']],how='left', on='hhno')
# df_psrc.drop('hhno', axis=1, inplace=True)


# hadkids
###### FIXME###########
# Set to 0 for now
df_psrc['hadkids'] = 0

# bucketBin
df_psrc['bucketBin'] = 1

# originalPUMA
df_psrc['orginalPUMA'] = df_psrc['PUMA5'].copy()

# hmultiunit
df_psrc['hmultiunit'] = 1
df_psrc.loc[df_psrc['htypdwel'] == 1, 'hmultiunit'] = 0

In [84]:

len(df_psrc)

1605803

In [85]:
df_psrc = df_psrc.loc[:,~df_psrc.columns.duplicated()]
output_cols = np.intersect1d(df_mtc.columns, df_psrc.columns)

In [40]:
df_psrc[output_cols].to_csv(os.path.join(base_dir,r'example_psrc','data','households.csv'),index=False)

In [86]:
df_psrc[output_cols]

,BLDGSZ,HHID,HHT,NOC,PERSONS,PUMA5,SERIALNO,TAZ,TENURE,UNITTYPE,...,hinccat1,hinccat2,hmultiunit,hownrent,hsizecat,htypdwel,hunittype,hwrkrcat,income,workers
0,2.0,1,4.0,0.0,1,11802,2017000283165,3547,3.0,1,...,3.0,8.0,0,2,1,1.0,1,0.0,85000,1.0
1,2.0,2,4.0,0.0,1,11802,2017000212400,3547,3.0,1,...,2.0,5.0,0,2,1,1.0,1,0.0,50000,1.0
2,2.0,3,6.0,0.0,1,11802,2015000894035,3547,2.0,1,...,3.0,6.0,0,1,1,1.0,1,0.0,56000,0.0
3,7.0,4,6.0,0.0,1,11802,2013000875777,3547,2.0,1,...,3.0,6.0,1,1,1,2.0,1,0.0,51000,0.0
4,2.0,5,4.0,0.0,1,11802,2013000264636,3547,1.0,1,...,3.0,7.0,0,1,1,1.0,1,0.0,74050,1.0
5,1.0,6,4.0,0.0,1,11802,2014000434358,3547,2.0,1,...,3.0,7.0,1,1,1,3.0,1,0.0,74600,1.0
6,2.0,7,4.0,0.0,1,11802,2014001079977,3547,1.0,1,...,3.0,7.0,0,1,1,1.0,1,0.0,72000,1.0
7,2.0,8,4.0,0.0,1,11802,2013000631819,3547,1.0,1,...,3.0,8.0,0,1,1,1.0,1,0.0,80000,1.0
8,2.0,9,4.0,0.0,1,11802,2013001482586,3547,1.0,1,...,3.0,8.0,0,1,1,1.0,1,0.0,100000,1.0
9,1.0,10,4.0,0.0,1,11802,2014000434358,3547,2.0,1,...,3.0,7.0,1,1,1,3.0,1,0.0,74600,1.0


# Persons

In [143]:
hh_persons = h5py.File(r'R:\e2projects_two\SoundCast\Inputs\dev\landuse\2018\base_year\hh_and_persons.h5', 'r')
df_psrc = pd.DataFrame()
for col in hh_persons['Household'].keys():
    df_psrc[col] = hh_persons['Household'][col][:]
    
df_psrc_person = pd.DataFrame()
for col in hh_persons['Person'].keys():
    df_psrc_person[col] = hh_persons['Person'][col][:]
    
hh_persons.close()

In [144]:
seed_persons = pd.read_csv(r'R:\e2projects_two\SyntheticPopulation_2018\FINAL_adjusted_outputs\adjusted_seed_persons.csv')
# _seed_persons = pd.read_csv(r'R:\e2projects_two\SyntheticPopulation_2018\populationsim_inputs\data\seed_persons.csv')
# seed_persons = seed_persons.append(_seed_persons)

In [145]:
len(seed_persons)

151471

In [146]:
df_mtc_persons = pd.read_csv(os.path.join(base_dir,r'example_mtc','data','persons.csv'))

df_psrc_person['HHID'] = df_psrc_person['hhno'].copy()
df_psrc_person['household_id'] = df_psrc_person['hhno'].copy()
df_psrc_person['age'] = df_psrc_person['pagey'].copy()
df_psrc_person['sex'] = df_psrc_person['pgend'].copy()

df_psrc_person.loc[df_psrc_person['pwtyp'] == 1,'pemploy'] = 1
df_psrc_person.loc[df_psrc_person['pwtyp'] == 2,'pemploy'] = 2
df_psrc_person.loc[df_psrc_person['pwtyp'] == 0,'pemploy'] = 3
df_psrc_person.loc[df_psrc_person['age'] < 16,'pemploy'] = 4
df_psrc_person['pemploy'] = df_psrc_person['pemploy'].astype('int')

df_psrc_person['pstudent'] = 3
df_psrc_person.loc[df_psrc_person['pptyp'].isin([7,6]),'pstudent'] = 1
df_psrc_person.loc[df_psrc_person['pptyp']==5 ,'pstudent'] = 2

df_psrc_person['ptype'] = df_psrc_person['pptyp']
df_psrc_person.loc[df_psrc_person['pptyp'] == 3, 'ptype'] = 5
df_psrc_person.loc[df_psrc_person['pptyp'] == 5, 'ptype'] = 3


# Join seed hh data for required columns
# seed_person_cols = ['hhnum','SPORDER']
# for col in seed_persons.columns:
#     if col.lower() in [i.lower() for i in df_mtc_persons.columns]:
#         seed_person_cols.append(col)
#     if col.lower() in df_mtc

In [142]:
# df_psrc_person['pno']

In [130]:
# Join synthetic household file to get the HHNUM to correspond to original PUMS data
# df_psrc_person = df_psrc_person.merge(syn_hh[['hhnum','household_id']], on='household_id', how='left')

In [118]:
# seed_persons

In [119]:
len(df_psrc_person)

4040013

In [121]:
# df_psrc_person[['hhnum','pno']].max()

In [122]:
seed_persons['SPORDER'].max()

20

In [131]:
# Join on hhnum and person ID
# df_psrc_person = df_psrc_person.merge(seed_persons[seed_person_cols], left_on=['hhnum','pno'], right_on=['hhnum','SPORDER'], how='left')

# FIX ME!!!!!!!!!!!!!!!!!!!!!!!!!!
# some synthetic persons don't have a corresponding house/pno and are excluded (<100,000)
# need to figure out why and add them in there

In [136]:
# df_psrc_person[df_psrc_person['SEX'].isnull()]

In [128]:
# df_psrc_person.rename(columns={'SPORDER': 'PNUM'}, inplace=True)
# df_psrc_person['PNUM'] = df_psrc_person['PNUM'].astype('int')

In [154]:
df_psrc_person[df_psrc_person['household_id'] ==1605264]

,DDP,EARNS,ESR,GRADE,HOURS,MSP,PAUG,PERID,PNUM,POVERTY,RELATE,WEEKS,age,household_id,padkid,pagecat,pemploy,pstudent,ptype,sex
4038901,-1,-1,-1,-1,-1,-1,-1,4038901,20,-1,-1,-1,59,1605264,-1,-1,3,3,4,2
4038915,-1,-1,-1,-1,-1,-1,-1,4038915,17,-1,-1,-1,12,1605264,-1,-1,4,1,7,1
4039177,-1,-1,-1,-1,-1,-1,-1,4039177,10,-1,-1,-1,66,1605264,-1,-1,3,3,5,1
4039466,-1,-1,-1,-1,-1,-1,-1,4039466,15,-1,-1,-1,18,1605264,-1,-1,3,1,6,2
4039485,-1,-1,-1,-1,-1,-1,-1,4039485,11,-1,-1,-1,17,1605264,-1,-1,3,1,6,2
4039537,-1,-1,-1,-1,-1,-1,-1,4039537,19,-1,-1,-1,14,1605264,-1,-1,4,1,7,1
4039665,-1,-1,-1,-1,-1,-1,-1,4039665,14,-1,-1,-1,60,1605264,-1,-1,3,3,4,2
4039708,-1,-1,-1,-1,-1,-1,-1,4039708,12,-1,-1,-1,16,1605264,-1,-1,3,1,6,2
4039833,-1,-1,-1,-1,-1,-1,-1,4039833,9,-1,-1,-1,31,1605264,-1,-1,3,3,4,2
4039924,-1,-1,-1,-1,-1,-1,-1,4039924,16,-1,-1,-1,10,1605264,-1,-1,4,1,7,1


In [149]:
df_psrc_person['PNUM'] = df_psrc_person['pno']

In [150]:
# Drop columns not in the original MTC data
for col in df_mtc_persons.columns:
    if col not in df_psrc_person.columns:
        df_psrc_person[col] = -1

output_cols = list(np.intersect1d(df_mtc_persons.columns, df_psrc_person.columns))
df_psrc_person['PERID'] = range(len(df_psrc_person))
# df_psrc_person = df_psrc_person[~df_psrc_person.duplicated()]
df_psrc_person = df_psrc_person[output_cols]

# Make sure data types are consistent between two data sets
for col in df_psrc_person.columns:
    if df_psrc_person[col].dtype != df_mtc_persons[col].dtype:
        df_psrc_person[col] = df_psrc_person[col].astype(df_mtc_persons[col].dtype)



C:\Anaconda\envs\model\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [152]:
# df_psrc_person

In [155]:
df_psrc_person.to_csv(os.path.join(base_dir,r'example_psrc','data','persons.csv'),index=False)

In [17]:
df_psrc_person = pd.read_csv(os.path.join(base_dir,r'example_psrc','data','persons.csv'))
df_psrc = pd.read_csv(os.path.join(base_dir,r'example_psrc','data','households.csv'))

In [151]:
len(df_psrc_person)

3984733

In [64]:
df_psrc_person['PERID'] = range(len(df_psrc_person))
df_psrc_person[df_psrc_person['PERID'].duplicated()]

,hhno,pagey,pdairy,pgend,pno,ppaidprk,pptyp,psexpfac,pspcl,pstaz,...,GRADE,PNUM,PAUG,DDP,WEEKS,HOURS,POVERTY,EARNS,pagecat,padkid


In [10]:
df_psrc[df_psrc['HHID'] == 45218]

,BLDGSZ,HHID,HHT,NOC,PERSONS,PUMA5,SERIALNO,TAZ,TENURE,UNITTYPE,...,hinccat1,hinccat2,hmultiunit,hownrent,hsizecat,htypdwel,hunittype,hwrkrcat,income,workers
45217,2.0,45218,4.0,0.0,1,11802,2015000770629,3601,1.0,1,...,2.0,5.0,0,1,1,1.0,1,0.0,45000,1.0


In [67]:
df_psrc_person[df_psrc_person['household_id'] == 45218]

,hhno,pagey,pdairy,pgend,pno,ppaidprk,pptyp,psexpfac,pspcl,pstaz,...,PNUM,PAUG,DDP,WEEKS,HOURS,MSP,POVERTY,EARNS,pagecat,padkid
2157725,45218,59,-1,1,1,-1,1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [100]:
df_psrc_person['PNUM']

0          1.0
1          1.0
2          1.0
3          1.0
4          1.0
          ... 
4040008    1.0
4040009    1.0
4040010    2.0
4040011    1.0
4040012    2.0
Name: PNUM, Length: 4040013, dtype: float64

In [74]:
seed_persons['SPORDER']

0         1
1         2
2         1
3         2
4         3
         ..
184808    1
184809    1
184810    1
184811    1
184812    1
Name: SPORDER, Length: 184813, dtype: int64

# Land Use

In [4]:
# Read household files
# Using additional data from person files not included in final output

df_psrc_person = pd.read_csv(os.path.join(base_dir,r'example_psrc','data','persons.csv'))
df_psrc = pd.read_csv(os.path.join(base_dir,r'example_psrc','data','households.csv'))
# df_psrc = pd.read_csv(os.path.join(base_dir,r'example_psrc','data','households.csv'))

In [5]:
df_mtc_lu = pd.read_csv(r'C:\Users\bnichols\cmd_acsim\test_example_mtc\example_mtc_full\data\land_use.csv')

In [5]:

df_mtc_lu.columns

Index([u'ZONE', u'DISTRICT', u'SD', u'COUNTY', u'TOTHH', u'HHPOP', u'TOTPOP',
       u'EMPRES', u'SFDU', u'MFDU', u'HHINCQ1', u'HHINCQ2', u'HHINCQ3',
       u'HHINCQ4', u'TOTACRE', u'RESACRE', u'CIACRE', u'SHPOP62P', u'TOTEMP',
       u'AGE0004', u'AGE0519', u'AGE2044', u'AGE4564', u'AGE65P', u'RETEMPN',
       u'FPSEMPN', u'HEREMPN', u'OTHEMPN', u'AGREMPN', u'MWTEMPN', u'PRKCST',
       u'OPRKCST', u'area_type', u'HSENROLL', u'COLLFTE', u'COLLPTE',
       u'TOPOLOGY', u'TERMINAL', u'ZERO', u'hhlds', u'sftaz', u'gqpop'],
      dtype='object')

In [6]:
df_parcel = pd.read_csv(r'R:\e2projects_two\SoundCast\Inputs\dev\landuse\2018\base_year\parcels_urbansim.txt',
                       delim_whitespace=True)

In [7]:
# # Load parcel geography lookup
# from sqlalchemy import create_engine
# conn = create_engine(r'sqlite:///R:/e2projects_two/SoundCast/Inputs/dev/db/soundcast_inputs.db')
# parcel_geog = pd.read_sql("SELECT * FROM parcel_2018_geography", con=conn)

In [8]:
df_parcel.columns

Index([u'aparks', u'empedu_p', u'empfoo_p', u'empgov_p', u'empind_p',
       u'empmed_p', u'empofc_p', u'empoth_p', u'empret_p', u'emprsc_p',
       u'empsvc_p', u'emptot_p', u'hh_p', u'lutype_p', u'nparks', u'parcelid',
       u'parkdy_p', u'parkhr_p', u'ppricdyp', u'pprichrp', u'sqft_p',
       u'stugrd_p', u'stuhgh_p', u'stuuni_p', u'taz_p', u'xcoord_p',
       u'ycoord_p'],
      dtype='object')

In [9]:
df_lu = df_parcel.groupby('taz_p').sum()[['hh_p','emptot_p','empret_p','stuhgh_p',
                                 'stuuni_p']]
df_lu.rename(columns={'hh_p': 'TOTHH','emptot_p': 'TOTEMP','empret_p': 'RETEMPN',
                     'stuhgh_p': 'HSENROLL', 'stuuni_p': 'COLLFTE'}, inplace=True)
df_lu = df_lu.reset_index()
df_lu['ZONE'] = df_lu['taz_p']

In [11]:
# Total population by TAZ
df = df_psrc.groupby('TAZ').sum()['PERSONS'].reset_index()
df.rename(columns={'PERSONS': 'TOTPOP'}, inplace=True)
df_lu = df_lu.merge(df, left_on='ZONE', right_on='TAZ')
df_lu.drop('TAZ',axis=1,inplace=True)
df_lu['HHPOP'] = df_lu['TOTPOP'].copy()

In [12]:
df_psrc_person.household_id.min()

1

In [13]:
df_psrc_person.columns

Index([u'DDP', u'EARNS', u'ESR', u'GRADE', u'HOURS', u'MSP', u'PAUG', u'PERID',
       u'PNUM', u'POVERTY', u'RELATE', u'WEEKS', u'age', u'household_id',
       u'padkid', u'pagecat', u'pemploy', u'pstudent', u'ptype', u'sex'],
      dtype='object')

In [14]:
# df_mtc['HHID']

In [15]:
# MPRES
# employed residents
df_psrc_person['HHID'] = df_psrc_person['household_id'].copy()
df_psrc_person['EMPRES'] = 0
df_psrc_person.loc[df_psrc_person['pemploy'].isin([1,2]),'EMPRES'] = 1

df = df_psrc_person.groupby('HHID').sum()[['EMPRES']].reset_index()
df_psrc = df_psrc.merge(df, on='HHID')
df_lu = df_lu.merge(df_psrc.groupby('TAZ').sum()[['EMPRES']].reset_index(), left_on='ZONE', right_on='TAZ', how='left')
df_lu.drop('TAZ', axis=1, inplace=True)

# SFDU (single family dwelling units, not used)
df_lu['SFDU'] = -1
df_lu['MFDU'] = -1


In [16]:
# Households in the lowest income quartile (less than $30,000 annually in $2000)
for colname in ['HHINCQ1','HHINCQ2','HHINCQ3','HHINCQ4']:
    df_psrc[colname] = 0

df_psrc.loc[df_psrc['income'] < 30000, 'HHINCQ1'] = 1
df_psrc.loc[(df_psrc['income'] >= 30000) & (df_psrc['income'] < 60000), 'HHINCQ2'] = 1
df_psrc.loc[(df_psrc['income'] >= 60000) & (df_psrc['income'] < 100000), 'HHINCQ3'] = 1
df_psrc.loc[df_psrc['income'] >= 100000, 'HHINCQ4'] = 1

In [17]:
df = df_psrc.groupby('TAZ').sum()[['HHINCQ1','HHINCQ2','HHINCQ3','HHINCQ4']].reset_index()
df_lu = df_lu.merge(df, left_on='ZONE', right_on='TAZ', how='left')
df_lu.drop('TAZ', axis=1, inplace=True)

In [18]:
# Total acres

df = (df_parcel.groupby('taz_p').sum()[['sqft_p']]/43560).reset_index()
df.rename(columns={'sqft_p': 'TOTACRE'}, inplace=True)
df_lu = df_lu.merge(df, on='taz_p')

In [19]:
# Some TAZs have 0 TOTACRE fields.
# Popuilate with regional average
df_lu.loc[df_lu['TOTACRE'] == 0, 'TOTACRE'] = df_lu.TOTACRE.mean()

In [20]:
# Acreage occupied by residential development - don't know if we have that
# For now scale it by population and employment, with a default of 1 for RES
df_lu['RESACRE'] = df_lu['TOTACRE']*(df_lu['TOTPOP']/(df_lu['TOTPOP']+df_lu['TOTEMP']))

# commercial acreage, set to 0.5 for now
df_lu['CIACRE'] = df_lu['TOTACRE']*(df_lu['TOTEMP']/(df_lu['TOTPOP']+df_lu['TOTEMP']))

In [21]:
df_lu.loc[df_lu['TOTHH'] == 0], 'TOTPOP','TOTHH'

(Empty DataFrame
 Columns: [taz_p, TOTHH, TOTEMP, RETEMPN, HSENROLL, COLLFTE, ZONE, TOTPOP, HHPOP, EMPRES, SFDU, MFDU, HHINCQ1, HHINCQ2, HHINCQ3, HHINCQ4, TOTACRE, RESACRE, CIACRE]
 Index: [], 'TOTPOP', 'TOTHH')

In [22]:
df_lu[df_lu['taz_p'] == 275][['RESACRE','CIACRE','TOTPOP','TOTEMP','TOTACRE']]

,RESACRE,CIACRE,TOTPOP,TOTEMP,TOTACRE
268,16.980547,12.719269,526,394,29.699816


In [23]:
# See what the MTC LU file looks like
df_mtc_lu[['RESACRE','CIACRE']].head()

,RESACRE,CIACRE
0,1.0,15.00000
1,1.0,24.79297
2,1.0,2.31799
3,1.0,18.00000
4,1.0,15.00000


In [24]:
df_mtc_lu[['RESACRE','CIACRE']].max()

RESACRE    13647.0
CIACRE      5397.0
dtype: float64

In [25]:
df_lu['household_density'] = df_lu.TOTHH / (df_lu.RESACRE + df_lu.CIACRE)
df_lu['employment_density'] = df_lu.TOTEMP / (df_lu.RESACRE + df_lu.CIACRE)
df_lu['density_index'] = (df_lu['household_density'] *df_lu['employment_density']) / (df_lu['household_density'] + df_lu['employment_density']).clip(lower=1)

In [26]:
# df_lu.to_csv(r'C:\Users\bnichols\lu.csv')

In [27]:
household_density = df_lu.TOTHH / (df_lu.RESACRE + df_lu.CIACRE)
employment_density = df_lu.TOTEMP / (df_lu.RESACRE + df_lu.CIACRE)
_df = (household_density *employment_density) / (household_density + employment_density).clip(lower=1)
_df[_df.isnull()]

Series([], dtype: float64)

In [28]:
household_density.mean()

27.839794439744054

In [29]:
df_mtc_lu['household_density'] = df_mtc_lu.TOTHH / (df_mtc_lu.RESACRE + df_mtc_lu.CIACRE)
df_mtc_lu['employment_density'] = df_mtc_lu.TOTEMP / (df_mtc_lu.RESACRE + df_mtc_lu.CIACRE)
df_mtc_lu['density_index'] = (df_mtc_lu['household_density'] *df_mtc_lu['employment_density']) / (df_mtc_lu['household_density'] + df_mtc_lu['employment_density']).clip(lower=1)

In [30]:
df_mtc_lu[df_mtc_lu['TAZ'] == 715]

,TAZ,DISTRICT,SD,COUNTY,TOTHH,TOTPOP,TOTACRE,RESACRE,CIACRE,TOTEMP,...,OPRKCST,area_type,HSENROLL,COLLFTE,COLLPTE,TOPOLOGY,TERMINAL,household_density,employment_density,density_index
714,715,15,15,4,2759,8461,137540.0,4636.09473,2964.0,17874,...,0.0,4,0.0,0.0,0.0,1,1.01749,0.363022,2.351813,0.314479


In [31]:
df_mtc_lu['density_index'].mean()

4.3551376664456702

In [32]:
df_lu.columns

Index([u'taz_p', u'TOTHH', u'TOTEMP', u'RETEMPN', u'HSENROLL', u'COLLFTE',
       u'ZONE', u'TOTPOP', u'HHPOP', u'EMPRES', u'SFDU', u'MFDU', u'HHINCQ1',
       u'HHINCQ2', u'HHINCQ3', u'HHINCQ4', u'TOTACRE', u'RESACRE', u'CIACRE',
       u'household_density', u'employment_density', u'density_index'],
      dtype='object')

In [33]:
# Share of population 62 or older
df_psrc_person.loc[df_psrc_person['age'] >= 62, 'SHPOP62P'] = 1
df = df_psrc_person.groupby('household_id').sum()['SHPOP62P'].fillna(0).reset_index()
df_psrc = df_psrc.merge(df, left_on='HHID', right_on='household_id')
df = df_psrc.groupby('TAZ').sum()['SHPOP62P'].reset_index()
df_lu = df_lu.merge(df, left_on='ZONE', right_on='TAZ')
df_lu['SHPOP62P'] = df_lu['SHPOP62P']/df_lu['TOTPOP']

In [34]:
df_psrc_person.loc[df_psrc_person['age'] < 5, 'AGE0004'] = 1
df_psrc_person.loc[(df_psrc_person['age'] >= 5) & (df_psrc_person['age'] < 20), 'AGE0519'] = 1
df_psrc_person.loc[(df_psrc_person['age'] >= 20) & (df_psrc_person['age'] < 45), 'AGE2044'] = 1
df_psrc_person.loc[(df_psrc_person['age'] >= 45) & (df_psrc_person['age'] < 65), 'AGE4564'] = 1
df_psrc_person.loc[df_psrc_person['age'] >= 65, 'AGE65P'] = 1

for colname in ['AGE0004','AGE0519','AGE2044','AGE4564','AGE65P']:
    df = df_psrc_person.groupby('household_id').sum()[colname].fillna(0).reset_index()
    df_psrc = df_psrc.merge(df, left_on='HHID', right_on='household_id')
    df = df_psrc.groupby('TAZ').sum()[colname].reset_index()
    df_lu = df_lu.merge(df, left_on='ZONE', right_on='TAZ')
    if 'TAZ' in df_lu.columns:
        df_lu.drop('TAZ', axis=1, inplace=True)

In [35]:
# PRKCST Hourly parking rate paid by long-term hours (8 hours)

# Take the average for all parcels (weighted by number of spaces)
df_parcel['weighted_daily_price'] = df_parcel['parkdy_p']*df_parcel['ppricdyp']
df = df_parcel.groupby('taz_p').sum()['weighted_daily_price']/df_parcel.groupby('taz_p').sum()['parkdy_p']

# Convert the daily total rate to hourly assuming 8 hours
df = df/8
df = pd.DataFrame(df,columns=['PRKCST']).reset_index().fillna(0)

df_lu = df_lu.merge(df, on='taz_p')

# OPRKCST Hourly parking rate paid by short-term parkers
df_parcel['weighted_hourly_price'] = df_parcel['parkhr_p']*df_parcel['pprichrp']
df = df_parcel.groupby('taz_p').sum()['weighted_hourly_price']/df_parcel.groupby('taz_p').sum()['parkhr_p']

# Convert the daily total rate to hourly assuming 8 hours
df = df/8
df = pd.DataFrame(df,columns=['OPRKCST']).reset_index().fillna(0)

df_lu = df_lu.merge(df, on='taz_p')

 

In [36]:
df_lu.columns

Index([u'taz_p', u'TOTHH', u'TOTEMP', u'RETEMPN', u'HSENROLL', u'COLLFTE',
       u'ZONE', u'TOTPOP', u'HHPOP', u'EMPRES', u'SFDU', u'MFDU', u'HHINCQ1',
       u'HHINCQ2', u'HHINCQ3', u'HHINCQ4', u'TOTACRE', u'RESACRE', u'CIACRE',
       u'household_density', u'employment_density', u'density_index', u'TAZ_x',
       u'SHPOP62P', u'TAZ_y', u'AGE0004', u'AGE0519', u'AGE2044', u'AGE4564',
       u'AGE65P', u'PRKCST', u'OPRKCST'],
      dtype='object')

In [ ]:
taz_df["density"] = \
    (taz_df.totpop + (2.5 * taz_df.totemp)) / taz_df.totacre
taz_df["areatype"] = pd.cut(
    taz_df.density,
    bins=[0, 6, 30, 55, 100, 300, np.inf],
    labels=[5, 4, 3, 2, 1, 0]
)

In [53]:
df_lu["density"] = (df_lu.TOTPOP + (2.5 * df_lu.TOTEMP)) / df_lu.TOTACRE
df_lu["area_type"] = pd.cut(
    df_lu.density,
    bins=[0, 6, 30, 55, 100, 300, np.inf],
    labels=[5, 4, 3, 2, 1, 0])

In [44]:
# FIXME!!!!!!!!!!!!!
# AREATYPE area type designation

# HSENROLL high school enrollment
# df_psrc_person.loc[(df_psrc_person['pstyp'] == 6), 'HSENROLL'] = 1
# df = df_psrc_person.groupby('hhno').sum()['HSENROLL'].fillna(0).reset_index()
# df_psrc = df_psrc.merge(df, left_on='HHID', right_on='hhno')
# df = df_psrc.groupby('TAZ').sum()['HSENROLL'].reset_index()
# df_lu = df_lu.merge(df, left_on='ZONE', right_on='TAZ')

# # COLLFTE full time college student
# df_psrc_person.loc[(df_psrc_person['pstyp'] == 1) & (df_psrc_person['pptyp'] == 5), 'COLLFTE'] = 1
# df = df_psrc_person.groupby('hhno').sum()['COLLFTE'].fillna(0).reset_index()
# df_psrc = df_psrc.merge(df, left_on='HHID', right_on='hhno')
# df = df_psrc.groupby('TAZ').sum()['COLLFTE'].reset_index()
# df_lu = df_lu.merge(df, left_on='ZONE', right_on='TAZ')

# # COLLPTE
# df_psrc_person.loc[(df_psrc_person['pstyp'] == 2) & (df_psrc_person['pptyp'] == 5), 'COLLPTE'] = 1
# df = df_psrc_person.groupby('hhno').sum()['COLLPTE'].fillna(0).reset_index()
# df_psrc = df_psrc.merge(df, left_on='HHID', right_on='hhno')
# df = df_psrc.groupby('TAZ').sum()['COLLPTE'].reset_index()
# df_lu = df_lu.merge(df, left_on='ZONE', right_on='TAZ')

# TERMINAL
# get terminal times from file
df_tt = pd.read_csv(r'https://raw.githubusercontent.com/psrc/soundcast/dev/inputs/model/intrazonals/destination_tt.in',
                   delim_whitespace=True, skiprows=4)

df_tt.columns = ['taz_p','TERMINAL']
df_tt['taz_p'] = df_tt['taz_p'].apply(lambda i: i.split(':')[0]).astype('int')
df_lu = df_lu.merge(df_tt, on='taz_p')

# TOPOLOGY
df_lu['TOPOLOGY'] = 1

# ZERO placeholder
df_lu['ZERO'] = 0

df_lu['SFTAZ'] = df_lu['taz_p']

# Group quarters population
df_lu['GQPOP'] = 0

In [60]:
for col in df_mtc_lu.columns:
    if col not in df_lu.columns:
        df_lu[col] = -1
df_lu['TAZ'] = df_lu['ZONE']
output_cols = output_cols = np.intersect1d(df_mtc_lu.columns, df_lu.columns)

df_lu[output_cols].to_csv(os.path.join(base_dir,r'example_psrc','data','land_use.csv'))

In [70]:
df_lu.head()

,taz_p,TOTHH,TOTEMP,RETEMPN,HSENROLL,COLLFTE,ZONE,TOTPOP,HHPOP,EMPRES,...,DISTRICT,SD,COUNTY,FPSEMPN,HEREMPN,OTHEMPN,AGREMPN,MWTEMPN,area_type,COLLPTE
0,1,385,0,0,0,0,1,973,973,474,...,-1,-1,-1,-1,-1,-1,-1,-1,5,-1
1,2,827,543,11,0,0,2,1724,1724,785,...,-1,-1,-1,-1,-1,-1,-1,-1,3,-1
2,3,1449,691,230,0,0,3,2711,2711,1302,...,-1,-1,-1,-1,-1,-1,-1,-1,3,-1
3,4,1433,574,153,0,0,4,2193,2193,1249,...,-1,-1,-1,-1,-1,-1,-1,-1,3,-1
4,5,302,412,50,0,0,5,714,714,464,...,-1,-1,-1,-1,-1,-1,-1,-1,3,-1


# Skims

In [1]:
import h5py

In [13]:
# Bike and walk skims
mtc_skims = h5py.File(r'R:\e2projects_two\activitysim\psrc_example\mtc_skims.h5')

In [14]:
# If skim name in mtc_skims, write to h5
h5out_path = r'D:\brice\skims.omx'
f = h5py.File(h5out_path, 'w')
f.create_group("data")

<HDF5 group "/data" (0 members)>

In [15]:
# mtc_skims['data'].keys()

In [16]:
# Bike and walk skims
myh5 = h5py.File(r'L:\RTP_2022\sc_2018_BASE\inputs\model\roster\5to6.h5')

results_dict = {}
matrix_size = len(myh5['Skims']['sov_inc2t'])

# Assuminig DIST is centroid-centroid distance...using walk distance
results_dict['DIST'] = myh5['Skims']['walkt'][:matrix_size,:matrix_size]*(3.0/60.0) # Assuming 3 mph walk speed
results_dict['DISTBIKE'] = myh5['Skims']['biket'][:matrix_size,:matrix_size]*(10.0/60.0) # Assuming 10 mph bike speed
results_dict['DISTWALK'] = myh5['Skims']['walkt'][:matrix_size,:matrix_size]*(3.0/60.0) # Assuming 10 mph bike speed

myh5 = h5py.File(r'L:\RTP_2022\sc_2018_BASE\inputs\model\roster\6to7.h5')

for tod in ['AM','MD','PM','EV','EA']:
    for mtc_mode in ['COM', 'EXP','HVY','LOC','LRF','TRN']:
    # Fare
        # Using AM fares for all time periods for now
        results_dict['WLK_'+mtc_mode+'_WLK_FAR__'+tod] = myh5['Skims']['mfafarbx'][:matrix_size,:matrix_size]/100.0
        results_dict['WLK_'+mtc_mode+'_DRV_FAR__'+tod] = myh5['Skims']['mfafarbx'][:matrix_size,:matrix_size]/100.0
        results_dict['DRV_'+mtc_mode+'_WLK_FAR__'+tod] = myh5['Skims']['mfafarbx'][:matrix_size,:matrix_size]/100.0
        
for skim_matrix in results_dict.keys():
    if skim_matrix in mtc_skims['data'].keys():
        print(skim_matrix)
        f.create_dataset("data/"+skim_matrix, data=results_dict[skim_matrix], compression='gzip')

del results_dict
del myh5
f.close()

DRV_HVY_WLK_FAR__EV
DRV_COM_WLK_FAR__PM
WLK_LOC_WLK_FAR__AM
WLK_HVY_DRV_FAR__EA
WLK_EXP_WLK_FAR__EV
WLK_EXP_WLK_FAR__MD
WLK_COM_WLK_FAR__AM
WLK_EXP_WLK_FAR__EA
WLK_HVY_DRV_FAR__EV
WLK_COM_DRV_FAR__PM
WLK_LOC_DRV_FAR__EV
DRV_LRF_WLK_FAR__PM
DRV_EXP_WLK_FAR__PM
WLK_LOC_DRV_FAR__EA
DRV_HVY_WLK_FAR__AM
DRV_COM_WLK_FAR__AM
WLK_LOC_WLK_FAR__PM
WLK_LOC_DRV_FAR__MD
DRV_LRF_WLK_FAR__AM
WLK_LRF_DRV_FAR__MD
DRV_LOC_WLK_FAR__EV
WLK_LRF_WLK_FAR__PM
DRV_LOC_WLK_FAR__EA
WLK_LRF_WLK_FAR__AM
WLK_HVY_WLK_FAR__AM
DISTBIKE
WLK_COM_WLK_FAR__PM
WLK_HVY_DRV_FAR__MD
DRV_HVY_WLK_FAR__MD
WLK_COM_DRV_FAR__EV
WLK_EXP_DRV_FAR__AM
WLK_LRF_DRV_FAR__EA
WLK_COM_DRV_FAR__MD
WLK_COM_DRV_FAR__EA
DRV_HVY_WLK_FAR__PM
DRV_LOC_WLK_FAR__MD
DRV_COM_WLK_FAR__MD
WLK_COM_WLK_FAR__MD
WLK_LOC_WLK_FAR__EV
WLK_HVY_WLK_FAR__PM
DRV_LRF_WLK_FAR__MD
DRV_HVY_WLK_FAR__EA
WLK_LRF_DRV_FAR__PM
WLK_LOC_WLK_FAR__EA
WLK_HVY_DRV_FAR__AM
WLK_EXP_WLK_FAR__AM
WLK_COM_WLK_FAR__EV
WLK_COM_WLK_FAR__EA
DRV_EXP_WLK_FAR__EV
WLK_LOC_DRV_FAR__AM
DRV_EXP_WLK

In [19]:
h5out_path = r'D:\brice\skims.omx'
f = h5py.File(h5out_path, 'a')

In [20]:
myh5 = h5py.File(r'L:\RTP_2022\sc_2018_BASE\inputs\model\roster\7to8.h5')
results_dict = {}
import numpy as np
# np.zeros(len())

zero_array = np.zeros((matrix_size,matrix_size))
# Vehicle matrices
#### DISTANCE ####
# Distance is only skimmed for once (7to8); apply to all TOD periods
for tod in ['AM','MD','PM','EV','EA']: # EA=early AM
    results_dict = {}
    
    results_dict['SOV_DIST__'+tod] = myh5['Skims']['sov_inc2d'][:matrix_size,:matrix_size]/100.0
    results_dict['HOV2_DIST__'+tod] = myh5['Skims']['hov2_inc2d'][:matrix_size,:matrix_size]/100.0
    results_dict['HOV3_DIST__'+tod] = myh5['Skims']['hov3_inc2d'][:matrix_size,:matrix_size]/100.0
    results_dict['SOVTOLL_DIST__'+tod] = myh5['Skims']['sov_inc2d'][:matrix_size,:matrix_size]/100.0
    results_dict['HOV2TOLL_DIST__'+tod] = myh5['Skims']['hov2_inc2d'][:matrix_size,:matrix_size]/100.0
    results_dict['HOV3TOLL_DIST__'+tod] = myh5['Skims']['hov3_inc2d'][:matrix_size,:matrix_size]/100.0

# Time
    results_dict['SOV_TIME__'+tod] = myh5['Skims']['sov_inc2t'][:matrix_size,:matrix_size]/100.0
    results_dict['HOV2_TIME__'+tod] = myh5['Skims']['hov2_inc2t'][:matrix_size,:matrix_size]/100.0
    results_dict['HOV3_TIME__'+tod] = myh5['Skims']['hov3_inc2t'][:matrix_size,:matrix_size]/100.0
    results_dict['SOVTOLL_TIME__'+tod] = myh5['Skims']['sov_inc2t'][:matrix_size,:matrix_size]/100.0
    results_dict['HOV2TOLL_TIME__'+tod] = myh5['Skims']['hov2_inc2t'][:matrix_size,:matrix_size]/100.0
    results_dict['HOV3TOLL_TIME__'+tod] = myh5['Skims']['hov3_inc2t'][:matrix_size,:matrix_size]/100.0

# Cost 
# BTOLL is bridge toll
# Set this to 0 and only use the value toll (?)
    results_dict['SOV_BTOLL__'+tod] = zero_array
    results_dict['HOV2_BTOLL__'+tod] = zero_array
    results_dict['HOV3_BTOLL__'+tod] = zero_array
    results_dict['SOVTOLL_BTOLL__'+tod] = zero_array
    results_dict['HOV2TOLL_BTOLL__'+tod] = zero_array
    results_dict['HOV3TOLL_BTOLL__'+tod] = zero_array

# VTOLL is value toll, assuming a per-mile price?
#     results_dict['SOV_VTOLL__'+tod] = myh5['Skims']['sov_inc2c'][:matrix_size,:matrix_size]
#     results_dict['HOV2_VTOLL__'+tod] = myh5['Skims']['hov2_inc2c'][:matrix_size,:matrix_size]
#     results_dict['HOV3_VTOLL__'+tod] = myh5['Skims']['hov3_inc2c'][:matrix_size,:matrix_size]
    results_dict['SOVTOLL_VTOLL__'+tod] = myh5['Skims']['sov_inc2c'][:matrix_size,:matrix_size]/100.0
    results_dict['HOV2TOLL_VTOLL__'+tod] = myh5['Skims']['hov2_inc2c'][:matrix_size,:matrix_size]/100.0
    results_dict['HOV3TOLL_VTOLL__'+tod] = myh5['Skims']['hov3_inc2c'][:matrix_size,:matrix_size]/100.0
    
    for skim_matrix in results_dict.keys():
        if skim_matrix in mtc_skims['data'].keys():
            print(skim_matrix)
            if skim_matrix not in f['data'].keys():
                f.create_dataset("data/"+skim_matrix, data=results_dict[skim_matrix], compression='gzip')

    del results_dict
del myh5
f.close()

HOV3_DIST__AM
SOVTOLL_BTOLL__AM
HOV3TOLL_BTOLL__AM
HOV2TOLL_BTOLL__AM
SOVTOLL_VTOLL__AM
HOV2_TIME__AM
HOV3TOLL_TIME__AM
HOV3_BTOLL__AM
SOV_BTOLL__AM
HOV2TOLL_DIST__AM
HOV2TOLL_TIME__AM
SOVTOLL_TIME__AM
HOV3_TIME__AM
SOV_TIME__AM
SOVTOLL_DIST__AM
HOV3TOLL_VTOLL__AM
HOV2_BTOLL__AM
HOV2TOLL_VTOLL__AM
SOV_DIST__AM
HOV3TOLL_DIST__AM
HOV2_DIST__AM
HOV3TOLL_DIST__MD
HOV2TOLL_TIME__MD
HOV2_DIST__MD
HOV2TOLL_DIST__MD
SOVTOLL_BTOLL__MD
SOV_TIME__MD
HOV3TOLL_VTOLL__MD
SOVTOLL_DIST__MD
SOVTOLL_TIME__MD
HOV3_DIST__MD
HOV3TOLL_BTOLL__MD
HOV3TOLL_TIME__MD
HOV3_BTOLL__MD
SOVTOLL_VTOLL__MD
SOV_BTOLL__MD
HOV2_TIME__MD
HOV2TOLL_VTOLL__MD
SOV_DIST__MD
HOV2TOLL_BTOLL__MD
HOV2_BTOLL__MD
HOV3_TIME__MD
HOV3_TIME__PM
HOV3TOLL_BTOLL__PM
SOVTOLL_TIME__PM
SOV_TIME__PM
HOV3TOLL_VTOLL__PM
SOVTOLL_DIST__PM
HOV2TOLL_BTOLL__PM
HOV2_DIST__PM
HOV2TOLL_TIME__PM
HOV2_BTOLL__PM
SOVTOLL_BTOLL__PM
SOV_DIST__PM
SOVTOLL_VTOLL__PM
HOV3TOLL_DIST__PM
HOV3_DIST__PM
HOV3TOLL_TIME__PM
HOV2TOLL_VTOLL__PM
HOV2TOLL_DIST__PM
SOV_BTOLL__

In [21]:
import os
import numpy as np
time_dict = {'AM': '7to8','MD': '10to14', 'PM': '17to18', 'EV': '18to20', 'EA': '5to6'}



# Averages taken from daysim outputs (see below)
# need to actually model this
access_dist = {'COM': 6.561, 'EXP': 6.948, 'HVY': 9.31, 'LOC':7.69 , 'LRF': 7.6803, 'TRN': 9.31}
access_time = {'COM': 16.195, 'EXP': 16.71, 'HVY': 20.66, 'LOC': 17.759, 'LRF': 15.892, 'TRN': 20.66}

egress_dist = {'COM': 6.338, 'EXP': 6.71, 'HVY': 9.04, 'LOC':7.41 , 'LRF': 7.321, 'TRN': 9.04}
egress_time = {'COM': 15.93, 'EXP': 15.936, 'HVY': 20.08, 'LOC': 17.479, 'LRF': 15.325, 'TRN': 20.08}

for tod, hour in time_dict.items():
    
    h5out_path = r'D:\brice\skims.omx'
    f = h5py.File(h5out_path, 'a')
    
    print(tod)
    print(hour)
    myh5 = h5py.File(os.path.join(r'L:\RTP_2022\sc_2018_BASE\inputs\model\roster',hour+'.h5'))

    submode_dict = {'COM':'c', 'EXP':'p', 'HVY':'r', 'LOC':'b','LRF':'f', 'TRN': 'r'}

    for mtc_mode, psrc_mode in submode_dict.items():
        results_dict = {}
        print(mtc_mode)
        if mtc_mode == 'LOC':
            # Walk Access Time
            results_dict['WLK_'+mtc_mode+'_WLK_WAUX__'+tod] = myh5['Skims']['auxwa'][:matrix_size,:matrix_size]/100.0
            results_dict['WLK_'+mtc_mode+'_DRV_WAUX__'+tod] = myh5['Skims']['auxwa'][:matrix_size,:matrix_size]/100.0
            results_dict['DRV_'+mtc_mode+'_WLK_WAUX__'+tod] = myh5['Skims']['auxwa'][:matrix_size,:matrix_size]/100.0
            
            # Initial Wait Time
            results_dict['WLK_'+mtc_mode+'_WLK_IWAIT__'+tod] = myh5['Skims']['iwtwa'][:matrix_size,:matrix_size]/100.0
            results_dict['WLK_'+mtc_mode+'_DRV_IWAIT__'+tod] = myh5['Skims']['iwtwa'][:matrix_size,:matrix_size]/100.0
            results_dict['DRV_'+mtc_mode+'_WLK_IWAIT__'+tod] = myh5['Skims']['iwtwa'][:matrix_size,:matrix_size]/100.0
            
            # Total Wait Time
            results_dict['WLK_'+mtc_mode+'_WLK_WAIT__'+tod] = myh5['Skims']['twtwa'][:matrix_size,:matrix_size]/100.0
            results_dict['WLK_'+mtc_mode+'_DRV_WAIT__'+tod] = myh5['Skims']['twtwa'][:matrix_size,:matrix_size]/100.0
            results_dict['DRV_'+mtc_mode+'_WLK_WAIT__'+tod] = myh5['Skims']['twtwa'][:matrix_size,:matrix_size]/100.0
            
            # In vehicle time for Walk access/egress
            results_dict['WLK_'+mtc_mode+'_WLK_TOTIVT__'+tod] = myh5['Skims']['ivtwa'][:matrix_size,:matrix_size]/100.0
            results_dict['WLK_'+mtc_mode+'_WLK_KEYIVT__'+tod] = myh5['Skims']['ivtwa'][:matrix_size,:matrix_size]/100.0
        
            # Transfer Time 
            results_dict['WLK_'+mtc_mode+'_WLK_XWAIT__'+tod] = myh5['Skims']['xfrwa'][:matrix_size,:matrix_size]/100.0
            results_dict['WLK_'+mtc_mode+'_DRV_XWAIT__'+tod] = myh5['Skims']['xfrwa'][:matrix_size,:matrix_size]/100.0
            results_dict['DRV_'+mtc_mode+'_WLK_XWAIT__'+tod] = myh5['Skims']['xfrwa'][:matrix_size,:matrix_size]/100.0
            
            # Boardings
            results_dict['WLK_'+mtc_mode+'_WLK_BOARDS__'+tod] = myh5['Skims']['ndbwa'][:matrix_size,:matrix_size]/100.0
            results_dict['WLK_'+mtc_mode+'_DRV_BOARDS__'+tod] = myh5['Skims']['ndbwa'][:matrix_size,:matrix_size]/100.0
            results_dict['DRV_'+mtc_mode+'_WLK_BOARDS__'+tod] = myh5['Skims']['ndbwa'][:matrix_size,:matrix_size]/100.0

            
        else:
            results_dict['WLK_'+mtc_mode+'_WLK_WAUX__'+tod] = myh5['Skims']['auxw'+psrc_mode][:matrix_size,:matrix_size]/100.0
            results_dict['WLK_'+mtc_mode+'_WLK_IWAIT__'+tod] = myh5['Skims']['iwtw'+psrc_mode][:matrix_size,:matrix_size]/100.0
            results_dict['WLK_'+mtc_mode+'_WLK_WAIT__'+tod] = myh5['Skims']['twtw'+psrc_mode][:matrix_size,:matrix_size]/100.0
            results_dict['WLK_'+mtc_mode+'_WLK_XWAIT__'+tod] = myh5['Skims']['xfrw'+psrc_mode][:matrix_size,:matrix_size]/100.0
            results_dict['WLK_'+mtc_mode+'_WLK_BOARDS__'+tod] = myh5['Skims']['ndbw'+psrc_mode][:matrix_size,:matrix_size]/100.0
            results_dict['WLK_'+mtc_mode+'_DRV_WAUX__'+tod] = myh5['Skims']['auxw'+psrc_mode][:matrix_size,:matrix_size]/100.0
            results_dict['WLK_'+mtc_mode+'_DRV_IWAIT__'+tod] = myh5['Skims']['iwtw'+psrc_mode][:matrix_size,:matrix_size]/100.0
            results_dict['WLK_'+mtc_mode+'_DRV_WAIT__'+tod] = myh5['Skims']['twtw'+psrc_mode][:matrix_size,:matrix_size]/100.0
            results_dict['WLK_'+mtc_mode+'_DRV_XWAIT__'+tod] = myh5['Skims']['xfrw'+psrc_mode][:matrix_size,:matrix_size]/100.0
            results_dict['WLK_'+mtc_mode+'_DRV_BOARDS__'+tod] = myh5['Skims']['ndbw'+psrc_mode][:matrix_size,:matrix_size]/100.0
            results_dict['DRV_'+mtc_mode+'_WLK_WAUX__'+tod] = myh5['Skims']['auxw'+psrc_mode][:matrix_size,:matrix_size]/100.0
            results_dict['DRV_'+mtc_mode+'_WLK_IWAIT__'+tod] = myh5['Skims']['iwtw'+psrc_mode][:matrix_size,:matrix_size]/100.0
            results_dict['DRV_'+mtc_mode+'_WLK_WAIT__'+tod] = myh5['Skims']['twtw'+psrc_mode][:matrix_size,:matrix_size]/100.0
            results_dict['DRV_'+mtc_mode+'_WLK_XWAIT__'+tod] = myh5['Skims']['xfrw'+psrc_mode][:matrix_size,:matrix_size]/100.0
            results_dict['DRV_'+mtc_mode+'_WLK_BOARDS__'+tod] = myh5['Skims']['ndbw'+psrc_mode][:matrix_size,:matrix_size]/100.0

        # In Vehicle Time
        
        results_dict['WLK_'+mtc_mode+'_DRV_TOTIVT__'+tod] = myh5['Skims']['ivtwa'+psrc_mode][:matrix_size,:matrix_size]/100.0
        results_dict['DRV_'+mtc_mode+'_WLK_TOTIVT__'+tod] = myh5['Skims']['ivtwa'+psrc_mode][:matrix_size,:matrix_size]/100.0
        
        # Modified In Vehicle Time
        # FIXME: look up how this is defined in TM1
        results_dict['DRV_'+mtc_mode+'_WLK_KEYIVT__'+tod] = myh5['Skims']['ivtwa'+psrc_mode][:matrix_size,:matrix_size]/100.0
        results_dict['WLK_'+mtc_mode+'_DRV_KEYIVT__'+tod] = myh5['Skims']['ivtwa'+psrc_mode][:matrix_size,:matrix_size]/100.0
                
        ####################
        # Drive to Transit #
        ####################
        
        # Drive access time
        results_dict['DRV_'+mtc_mode+'_WLK_DTIM__'+tod] = np.full([matrix_size,matrix_size], access_time[mtc_mode]) 
        results_dict['WLK_'+mtc_mode+'_DRV_DTIM__'+tod] = np.full([matrix_size,matrix_size], access_time[mtc_mode]) 
        
        # Drive access distance
        results_dict['DRV_'+mtc_mode+'_WLK_DDIST__'+tod] = np.full([matrix_size,matrix_size], egress_time[mtc_mode]) 
        results_dict['WLK_'+mtc_mode+'_DRV_DDIST__'+tod] = np.full([matrix_size,matrix_size], egress_time[mtc_mode])
        
        for skim_matrix in results_dict.keys():
            if skim_matrix in mtc_skims['data'].keys():
                print(skim_matrix)
                if skim_matrix not in f['data'].keys():
                    f.create_dataset("data/"+skim_matrix, data=results_dict[skim_matrix], compression='gzip')

        del results_dict
    
    results_dict = {}
    ##### Some other random stuff
    results_dict['DRV_LRF_WLK_FERRYIVT__'+tod] = myh5['Skims']['ivtwaf'][:matrix_size,:matrix_size]/100.0
    results_dict['WLK_LRF_WLK_FERRYIVT__'+tod] = myh5['Skims']['ivtwaf'][:matrix_size,:matrix_size]/100.0
    results_dict['WLK_LRF_DRV_FERRYIVT__'+tod] = myh5['Skims']['ivtwaf'][:matrix_size,:matrix_size]/100.0
    results_dict['WLK_COM_WLK_KEYIVT__'+tod] = myh5['Skims']['ivtwac'][:matrix_size,:matrix_size]/100.0
    results_dict['WLK_COM_WLK_TOTIVT__'+tod] = myh5['Skims']['ivtwac'][:matrix_size,:matrix_size]/100.0
    results_dict['WLK_EXP_WLK_KEYIVT__'+tod] = myh5['Skims']['ivtwap'][:matrix_size,:matrix_size]/100.0
    results_dict['WLK_EXP_WLK_TOTIVT__'+tod] = myh5['Skims']['ivtwap'][:matrix_size,:matrix_size]/100.0
    results_dict['WLK_HVY_WLK_KEYIVT__'+tod] = myh5['Skims']['ivtwar'][:matrix_size,:matrix_size]/100.0
    results_dict['WLK_HVY_WLK_TOTIVT__'+tod] = myh5['Skims']['ivtwar'][:matrix_size,:matrix_size]/100.0
    results_dict['WLK_LRF_WLK_KEYIVT__'+tod] = myh5['Skims']['ivtwaf'][:matrix_size,:matrix_size]/100.0
    results_dict['WLK_LRF_WLK_TOTIVT__'+tod] = myh5['Skims']['ivtwaf'][:matrix_size,:matrix_size]/100.0
    
    results_dict['WLK_TRN_WLK_IVT__'+tod] = myh5['Skims']['ivtwar'][:matrix_size,:matrix_size]/100.0
    results_dict['WLK_TRN_WLK_WAUX__'+tod] = myh5['Skims']['auxwa'][:matrix_size,:matrix_size]/100.0
    results_dict['WLK_TRN_WLK_WEGR__'+tod] = myh5['Skims']['auxwa'][:matrix_size,:matrix_size]/100.0
    results_dict['WLK_TRN_WLK_WACC__'+tod] = myh5['Skims']['auxwa'][:matrix_size,:matrix_size]/100.0
    
    for skim_matrix in results_dict.keys():
        if skim_matrix in mtc_skims['data'].keys():
            print(skim_matrix)
            if skim_matrix not in f['data'].keys():
                f.create_dataset("data/"+skim_matrix, data=results_dict[skim_matrix], compression='gzip')

    del results_dict
    del myh5
    
    f.close()

MD
10to14
LOC
DRV_LOC_WLK_WAUX__MD
WLK_LOC_WLK_IWAIT__MD
WLK_LOC_WLK_XWAIT__MD
WLK_LOC_DRV_BOARDS__MD
DRV_LOC_WLK_WAIT__MD
WLK_LOC_DRV_IWAIT__MD
WLK_LOC_DRV_WAIT__MD
WLK_LOC_DRV_WAUX__MD
WLK_LOC_DRV_XWAIT__MD
WLK_LOC_DRV_DDIST__MD
DRV_LOC_WLK_TOTIVT__MD
DRV_LOC_WLK_DTIM__MD
WLK_LOC_WLK_WAIT__MD
WLK_LOC_WLK_BOARDS__MD
DRV_LOC_WLK_BOARDS__MD
WLK_LOC_WLK_TOTIVT__MD
WLK_LOC_WLK_WAUX__MD
DRV_LOC_WLK_XWAIT__MD
WLK_LOC_DRV_TOTIVT__MD
WLK_LOC_DRV_DTIM__MD
DRV_LOC_WLK_DDIST__MD
DRV_LOC_WLK_IWAIT__MD
TRN
WLK_TRN_WLK_XWAIT__MD
WLK_TRN_WLK_IWAIT__MD
WLK_TRN_WLK_WAUX__MD
LRF
WLK_LRF_WLK_WAUX__MD
DRV_LRF_WLK_KEYIVT__MD
DRV_LRF_WLK_WAUX__MD
WLK_LRF_DRV_DDIST__MD
WLK_LRF_WLK_XWAIT__MD
WLK_LRF_DRV_KEYIVT__MD
DRV_LRF_WLK_WAIT__MD
WLK_LRF_DRV_TOTIVT__MD
DRV_LRF_WLK_BOARDS__MD
WLK_LRF_DRV_XWAIT__MD
DRV_LRF_WLK_TOTIVT__MD
DRV_LRF_WLK_DDIST__MD
WLK_LRF_DRV_WAIT__MD
DRV_LRF_WLK_XWAIT__MD
DRV_LRF_WLK_DTIM__MD
WLK_LRF_DRV_WAUX__MD
WLK_LRF_DRV_DTIM__MD
WLK_LRF_WLK_IWAIT__MD
WLK_LRF_DRV_BOARDS__MD
DRV_LRF_WLK_IW

In [23]:
f = h5py.File(h5out_path, 'a')

In [28]:
f['data']

KeyError: "Unable to open object (Object 'data' doesn't exist)"

In [73]:
# Get a list of skims that need to be created
[i for i in mtc_skims['data'].keys() if i not in results_dict.keys()]

[]

In [71]:
# mtc_skims['data']['WLK_LOC_WLK_KEYIVT__MD']

In [47]:
myh5 = h5py.File(r'L:\RTP_2022\sc_2018_BASE\inputs\model\roster\7to8.h5')

# Create the transit fields we can
# walk access and walk egress

# Boardings for transit walk access and egress
results_dict['WLK_COM_WLK_BOARDS__AM'] mfndbwa

# Fare is DRV_COM_WALK_FAR
# Not sure the difference between KEYIVT vs TOTIVT
# TOTIVT is the standard total in-vehicle time
# KEYIVT is described as "incremental w/ IVT"
# the coefficient for this is regular IVT-IVT(submode)
# BOARDS -> boardings
# IWAIT
# XWAIT





# All transit (?) local bus...?
#  "actual_in_vehicle_times": "mfivtwab"
# "actual_in_vehicle_times": "mfivtwap"
# "actual_in_vehicle_times": "mfivtwar"
#  "actual_in_vehicle_times": "mfivtwac"
# "actual_in_vehicle_times": "mfivtwaf"
# "actual_in_vehicle_times": "mfivtwa",
# "actual_aux_transit_times": "mfauxwa",
#  "avg_boardings": "mfndbwa"
# "actual_first_waiting_times": "mfiwtwa",
# "actual_total_waiting_times": "mftwtwa"

## Commuter Rail
# "actual_in_vehicle_times": "mfivtwcb"
# "actual_in_vehicle_times": "mfivtwcp" -> see above, but replace 2nd to last "a" with "c" for the mode
# c -> commuter rail
# f -> ferry
# r -> light rail
# w

# How to differentiate drive to transit

# DRV_COM_WLK_BOARDS__AM: Drive to commuter rail | drive access | walk egress



In [64]:
test = myh5['Skims']['ndbwc'][:]

In [68]:
myh5['Skims']['sov_inc2t'][:4]

array([[ 407,  389,  625, ...,  767, 4914, 6261],
       [ 388,  330,  375, ...,  664, 4871, 6185],
       [ 626,  389,  332, ...,  593, 4875, 6188],
       [ 556,  411,  422, ...,  487, 4698, 6011]], dtype=uint16)

In [66]:
test.max()

65535

In [ ]:
# myh5 = h5py.File(r'L:\RTP_2022\sc_2018_BASE\inputs\model\roster\5to6.h5')
# myh5['Skims'].keys()

# Create some drive-to-transit skims

In [63]:
# Drive distance DRV_LOC_WLK_DDIST
trip = pd.read_csv(r'Q:\sc_bike_dev_2018\outputs\daysim\_trip.tsv', delim_whitespace=True)

In [64]:
tour = pd.read_csv(r'Q:\sc_bike_dev_2018\outputs\daysim\_tour.tsv', delim_whitespace=True)

In [4]:
trip.groupby('pathtype').count()

,id,tour_id,hhno,pno,day,tour,half,tseg,tsvid,opurp,...,deptm,arrtm,endacttm,travtime,travcost,travdist,vot,trexpfac,od,sov_ff_time
pathtype,,,,,,,,,,,,,,,,,,,,,
0,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
1,15605496,15605496,15605496,15605496,15605496,15605496,15605496,15605496,15605496,15605496,...,15605496,15605496,15605496,15605496,15605496,15605496,15605496,15605496,15605496,15605496
3,377631,377631,377631,377631,377631,377631,377631,377631,377631,377631,...,377631,377631,377631,377631,377631,377631,377631,377631,377631,377631
4,64734,64734,64734,64734,64734,64734,64734,64734,64734,64734,...,64734,64734,64734,64734,64734,64734,64734,64734,64734,64734
5,3616,3616,3616,3616,3616,3616,3616,3616,3616,3616,...,3616,3616,3616,3616,3616,3616,3616,3616,3616,3616
6,16252,16252,16252,16252,16252,16252,16252,16252,16252,16252,...,16252,16252,16252,16252,16252,16252,16252,16252,16252,16252
7,10761,10761,10761,10761,10761,10761,10761,10761,10761,10761,...,10761,10761,10761,10761,10761,10761,10761,10761,10761,10761


In [5]:
tour.groupby('tpathtp').count()

,id,person_id,person_day_id,hhno,pno,day,tour,jtindex,parent,subtrs,...,tautotime,tautocost,tautodist,tripsh1,tripsh2,phtindx1,phtindx2,fhtindx1,fhtindx2,toexpfac
tpathtp,,,,,,,,,,,,,,,,,,,,,
0,202624,202624,202624,202624,202624,202624,202624,202624,202624,202624,...,202624,202624,202624,202624,202624,202624,202624,202624,202624,202624
1,5895523,5895523,5895523,5895523,5895523,5895523,5895523,5895523,5895523,5895523,...,5895523,5895523,5895523,5895523,5895523,5895523,5895523,5895523,5895523,5895523
3,182761,182761,182761,182761,182761,182761,182761,182761,182761,182761,...,182761,182761,182761,182761,182761,182761,182761,182761,182761,182761
4,47682,47682,47682,47682,47682,47682,47682,47682,47682,47682,...,47682,47682,47682,47682,47682,47682,47682,47682,47682,47682
5,1414,1414,1414,1414,1414,1414,1414,1414,1414,1414,...,1414,1414,1414,1414,1414,1414,1414,1414,1414,1414
6,18396,18396,18396,18396,18396,18396,18396,18396,18396,18396,...,18396,18396,18396,18396,18396,18396,18396,18396,18396,18396
7,7103,7103,7103,7103,7103,7103,7103,7103,7103,7103,...,7103,7103,7103,7103,7103,7103,7103,7103,7103,7103


In [6]:
tour.groupby('tmodetp').count()

,id,person_id,person_day_id,hhno,pno,day,tour,jtindex,parent,subtrs,...,tautotime,tautocost,tautodist,tripsh1,tripsh2,phtindx1,phtindx2,fhtindx1,fhtindx2,toexpfac
tmodetp,,,,,,,,,,,,,,,,,,,,,
0,340,340,340,340,340,340,340,340,340,340,...,340,340,340,340,340,340,340,340,340,340
1,712853,712853,712853,712853,712853,712853,712853,712853,712853,712853,...,712853,712853,712853,712853,712853,712853,712853,712853,712853,712853
2,157042,157042,157042,157042,157042,157042,157042,157042,157042,157042,...,157042,157042,157042,157042,157042,157042,157042,157042,157042,157042
3,2158266,2158266,2158266,2158266,2158266,2158266,2158266,2158266,2158266,2158266,...,2158266,2158266,2158266,2158266,2158266,2158266,2158266,2158266,2158266,2158266
4,1481664,1481664,1481664,1481664,1481664,1481664,1481664,1481664,1481664,1481664,...,1481664,1481664,1481664,1481664,1481664,1481664,1481664,1481664,1481664,1481664
5,1385358,1385358,1385358,1385358,1385358,1385358,1385358,1385358,1385358,1385358,...,1385358,1385358,1385358,1385358,1385358,1385358,1385358,1385358,1385358,1385358
6,222643,222643,222643,222643,222643,222643,222643,222643,222643,222643,...,222643,222643,222643,222643,222643,222643,222643,222643,222643,222643
7,34713,34713,34713,34713,34713,34713,34713,34713,34713,34713,...,34713,34713,34713,34713,34713,34713,34713,34713,34713,34713
8,202624,202624,202624,202624,202624,202624,202624,202624,202624,202624,...,202624,202624,202624,202624,202624,202624,202624,202624,202624,202624


In [8]:
df.groupby('dpurp').count()

,id,tour_id,hhno,pno,day,tour,half,tseg,tsvid,opurp,...,deptm,arrtm,endacttm,travtime,travcost,travdist,vot,trexpfac,od,sov_ff_time
dpurp,,,,,,,,,,,,,,,,,,,,,
0,34713,34713,34713,34713,34713,34713,34713,34713,34713,34713,...,34713,34713,34713,34713,34713,34713,34713,34713,34713,34713
1,38510,38510,38510,38510,38510,38510,38510,38510,38510,38510,...,38510,38510,38510,38510,38510,38510,38510,38510,38510,38510
2,233,233,233,233,233,233,233,233,233,233,...,233,233,233,233,233,233,233,233,233,233
3,1579,1579,1579,1579,1579,1579,1579,1579,1579,1579,...,1579,1579,1579,1579,1579,1579,1579,1579,1579,1579
4,2655,2655,2655,2655,2655,2655,2655,2655,2655,2655,...,2655,2655,2655,2655,2655,2655,2655,2655,2655,2655
5,3212,3212,3212,3212,3212,3212,3212,3212,3212,3212,...,3212,3212,3212,3212,3212,3212,3212,3212,3212,3212
6,1130,1130,1130,1130,1130,1130,1130,1130,1130,1130,...,1130,1130,1130,1130,1130,1130,1130,1130,1130,1130
7,1751,1751,1751,1751,1751,1751,1751,1751,1751,1751,...,1751,1751,1751,1751,1751,1751,1751,1751,1751,1751
10,69426,69426,69426,69426,69426,69426,69426,69426,69426,69426,...,69426,69426,69426,69426,69426,69426,69426,69426,69426,69426


SyntaxError: invalid syntax (<ipython-input-81-f4380288d80c>, line 1)

In [82]:
# Bus
df = trip[trip['tour_id'].isin(tour[(tour['tmodetp'] == 7)&(tour['tpathtp']==3)]['id'])]

# Access
print(df[(df['dpurp'] == 10) & (df['mode'].isin([3,4,5]))]['travdist'].mean())

# Access
print(df[(df['dpurp'] == 10) & (df['mode'].isin([3,4,5]))]['travtime'].mean())

# Egress
# Access
print(df[(df['opurp'] == 10) & (df['mode'].isin([3,4,5]))]['travdist'].mean())
print(df[(df['opurp'] == 10) & (df['mode'].isin([3,4,5]))]['travtime'].mean())

7.69223182366
17.7693072071
7.4118038943
17.4790585852


In [83]:
# LIGHT RAIL
df = trip[trip['tour_id'].isin(tour[(tour['tmodetp'] == 7)&(tour['tpathtp']==4)]['id'])]

# Access
print(df[(df['dpurp'] == 10) & (df['mode'].isin([3,4,5]))]['travdist'].mean())

# Access
print(df[(df['dpurp'] == 10) & (df['mode'].isin([3,4,5]))]['travtime'].mean())

# Egress
# Access
print(df[(df['opurp'] == 10) & (df['mode'].isin([3,4,5]))]['travdist'].mean())
print(df[(df['opurp'] == 10) & (df['mode'].isin([3,4,5]))]['travtime'].mean())

9.31058639616
20.6637907415
9.0455961413
20.0863478307


In [84]:
# EXPRESS/PREMIUM BUS
df = trip[trip['tour_id'].isin(tour[(tour['tmodetp'] == 7)&(tour['tpathtp']==5)]['id'])]

# Access
print(df[(df['dpurp'] == 10) & (df['mode'].isin([3,4,5]))]['travdist'].mean())

# Access
print(df[(df['dpurp'] == 10) & (df['mode'].isin([3,4,5]))]['travtime'].mean())

# Egress
# Access
print(df[(df['opurp'] == 10) & (df['mode'].isin([3,4,5]))]['travdist'].mean())
print(df[(df['opurp'] == 10) & (df['mode'].isin([3,4,5]))]['travtime'].mean())

6.94899779885
16.7139607069
6.71822663435
16.2687342807


In [85]:
# cOMMUTER RAIL
df = trip[trip['tour_id'].isin(tour[(tour['tmodetp'] == 7)&(tour['tpathtp']==6)]['id'])]

# Access
print(df[(df['dpurp'] == 10) & (df['mode'].isin([3,4,5]))]['travdist'].mean())

# Access
print(df[(df['dpurp'] == 10) & (df['mode'].isin([3,4,5]))]['travtime'].mean())

# Egress
# Access
print(df[(df['opurp'] == 10) & (df['mode'].isin([3,4,5]))]['travdist'].mean())
print(df[(df['opurp'] == 10) & (df['mode'].isin([3,4,5]))]['travtime'].mean())

6.56109357826
16.195817752
6.33838242155
15.9363890524


In [87]:
# Ferry
df = trip[trip['tour_id'].isin(tour[(tour['tmodetp'] == 7)&(tour['tpathtp']==7)]['id'])]

# Access
print(df[(df['dpurp'] == 10) & (df['mode'].isin([3,4,5]))]['travdist'].mean())

# Access
print(df[(df['dpurp'] == 10) & (df['mode'].isin([3,4,5]))]['travtime'].mean())

# Egress
# Access
print(df[(df['opurp'] == 10) & (df['mode'].isin([3,4,5]))]['travdist'].mean())
print(df[(df['opurp'] == 10) & (df['mode'].isin([3,4,5]))]['travtime'].mean())

7.68038876571
15.8926711357
7.32125664881
15.3258262221


In [14]:
df[(df['dpurp'] == 10) & (df['mode'].isin([3,4,5]))]['travdist'].max()

77.870000000000005

In [10]:
df[(df['dpurp'] == 10) & (df['mode'].isin([3,4,5]))]['travtime'].mean()

17.861600369550811

In [11]:
df[(df['dpurp'] == 10) & (df['mode'].isin([3,4,5]))]['travcost'].mean()

1.5894584987550766

In [68]:
# Drive Egress
df[(df['mode'].isin([3,4,5]))& (df['opurp'] == 10) ]['travdist'].mean()

7.453273957188516

In [69]:
# Drive Egress
df[(df['mode'].isin([3,4,5]))& (df['opurp'] == 10) ]['travtime'].mean()

17.551396871433713